In [ ]:
import os, sys

project_root = os.path.abspath(os.path.join(os.getcwd(), '..'))
os.chdir(project_root)
sys.path.append(project_root)

In [ ]:
import pandas as pd
import src.Preprocessing
from torch.utils.data import Dataset
dataset_train = pd.read_csv("data/pubmed_train.csv",nrows = 1000)

In [2]:

class DatasetMed(Preprocessing.PubMedTokenizerPipeline):
    def __init__(self,max_length):
        super().__init__(max_length=max_length)
    def users_fields(self, examples):
        assert "abstract" in examples
        return examples['abstract']

In [3]:
def preprocess(data):
    print("before \n",data.isna().sum())     
    return  data.dropna(axis=0)

In [4]:
def add_labels(example):
    example["labels"] = example["input_ids"].clone()
    return example

In [5]:
def Pipeline(data):  
    cl = preprocess(data)  
    tokenizer_pipeline = DatasetMed(max_length=256)
    tokenized_dataset = tokenizer_pipeline(cl)
    ds = tokenized_dataset.map(add_labels)
    columns_to_keep = ["input_ids", "attention_mask", "labels"]
    ds = ds.remove_columns([col for col in ds.column_names if col not in columns_to_keep])
    return ds

In [ ]:

ds = Pipeline(dataset_train)
ds.to_parquet("data/dataset_big.parquet")

before 
 article     1129
abstract       0
dtype: int64


Map:   0%|          | 0/48871 [00:00<?, ? examples/s]

Map:   0%|          | 0/48871 [00:00<?, ? examples/s]

Creating parquet from Arrow format:   0%|          | 0/49 [00:00<?, ?ba/s]

163229140